<a href="https://colab.research.google.com/github/AtulAravindDas/FinVAR_LLM_Version/blob/main/context_query_builder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-core langgraph>0.2.27

In [ ]:
%%writefile /content/drive/MyDrive/context_query_builder.py
from langchain_core.prompts import PromptTemplate
import numpy as np
import pandas as pd

class ContextQueryBuilder:
    def __init__(self, ticker, fmp_data, risk_statement):
        self.ticker = ticker
        self.fmp_data = fmp_data
        self.risk_statement = risk_statement

    def generate_query(self):
        query = f"How risky is it to invest in {self.ticker}"
        return query

    def generate_context(self):
        company_name = self.fmp_data["info"].get("longName", "N/A")
        industry = self.fmp_data["info"].get("industry", "N/A")
        pe_ratio = self.fmp_data["info"].get("trailingPE", "N/A")
        eps = self.fmp_data["info"].get("epsTrailingTwelveMonths", "N/A")
        beta = self.fmp_data["info"].get("beta", "N/A")
        description = self.fmp_data["info"].get("description", "N/A")
        market_cap = self.fmp_data["info"].get("marketCap", "N/A")
        total_revenue = self.fmp_data["financials"][0].get("revenue", "N/A")

        ratios = self.fmp_data["ratios"]

        context = f"""
        Company Name: {company_name}
        Ticker: {self.ticker}
        Industry: {industry}
        Market Cap: {market_cap}
        Total Revenue (TTM): {total_revenue}
        Description: {description}
        Trailing P/E Ratio: {pe_ratio}
        EPS (TTM): {eps}
        Beta: {beta}
        Risk Factors (from 10-K):
        {self.risk_statement[0]}
        Quantitative and Qualitative Description of Risk:
        {self.risk_statement[1]}
        """.strip()

        liquidity_solvency_block = f"""

        Liquidity and Solvency Ratios:
        - Current Ratio: {ratios.get('Current Ratio', 'N/A')}
        - Quick Ratio: {ratios.get('Quick Ratio', 'N/A')}
        - Debt-to-Asset Ratio: {ratios.get('Debt-to-Asset', 'N/A')}
        - Interest Coverage Ratio: {ratios.get('Interest Coverage Ratio', 'N/A')}
        - Free Cash Flow: {ratios.get('FCF', 'N/A')}
        - Revenue Growth: {ratios.get('Revenue Growth', 'N/A')}

        Financial Health Indicators:
        - Current Ratio >1.0 indicates company can cover short-term obligations
        - Quick Ratio >1.0 shows strong liquidity without relying on inventory
        - Lower Debt-to-Asset ratios generally indicate less financial leverage and risk
        - Higher Interest Coverage ratios show better ability to service debt
        - Positive Free Cash Flow shows company generates cash after necessary investments
        - Positive Revenue Growth indicates expanding business operations
        """.rstrip()

        context += liquidity_solvency_block
        return context

    def prompt_creation(self):
        context = self.generate_context()
        print("Context Generated Successfully")
        query = self.generate_query()

        template = """
You are a FINRA-certified financial analyst.

CRITICAL GROUNDING RULES — FOLLOW EXACTLY:
1) Use ONLY the information inside the <CONTEXT>…</CONTEXT> block.
2) Do NOT use outside knowledge. If something is missing, write it under **Missing Data** as `INSUFFICIENT_CONTEXT: <what>`.
3) Every analytical claim must reference a fact extracted from the context using [F#] tags.
4) Quotes must be verbatim snippets (≤10 words) copied from the context.

OUTPUT FORMAT (do not add extra sections):

### Key Facts (verbatim from context)
- [F1] "<short quote>"
- [F2] "<short quote>"
- [F3] "<short quote>"
(Add 3–8 facts total)

### Risk Assessment
Cover:
- Industry characteristics and competitive dynamics [reference facts via F-tags]
- Valuation ratios (P/E, EPS, Beta) [F-tags]
- Liquidity and solvency ratios (only if present) [F-tags or mark INSUFFICIENT_CONTEXT]
- Revenue and Net Income trends (only if present) [F-tags or mark INSUFFICIENT_CONTEXT]
- Risks from 10-K Risk Factors [F-tags]

### Final Risk Level
<Low | Medium | High>  (must be justified with F-tags)

### Missing Data
List each required item that is not present as:
INSUFFICIENT_CONTEXT: <item>

<CONTEXT>
{context}
</CONTEXT>

Question:
{query}
""".strip()

        prompt = PromptTemplate.from_template(template)
        formatted_prompt = prompt.format(context=context, query=query)
        return formatted_prompt

Overwriting /content/drive/MyDrive/context_query_builder.py
